In [4]:
import polymerist as ps
from polymerist import genutils, polymerist, duration, tests
from polymerist.genutils import bits, typetools, importutils
from polymerist.genutils.decorators.functional import allow_string_paths

from polymerist.genutils.pkginspect import is_module, is_package, get_dir_path_within_package, from_package

import json, math
import pkgutil, importlib

print(importutils.module_hierarchy(json))
import json, math
mods = (
    ps,
    polymerist, 
    genutils,
    duration,
    bits,
    typetools,
    json,
    tests,
    math,
)

for mod in mods:
    print(is_module(mod))
    print(mod.__name__)
    print(is_package(mod))
    print(from_package(mod))

    try:
        print(get_dir_path_within_package('typetools', mod))
    except Exception as e:
        print(f'FAILED: {e!s}')
    
    print('='*50)

├───decoder
├───encoder
├───scanner
└───tool
True
polymerist
True
/home/timber/Documents/Python/polymerist_dev/polymerist/build/__editable__.polymerist-1.0.0+542.g31f379e-py3-none-any/polymerist
FAILED: polymerist contains no resource "typetools"
True
polymerist.polymerist
False
/home/timber/Documents/Python/polymerist_dev/polymerist/build/__editable__.polymerist-1.0.0+542.g31f379e-py3-none-any/polymerist
FAILED: <module 'polymerist.polymerist' from '/home/timber/Documents/Python/polymerist_dev/polymerist/build/__editable__.polymerist-1.0.0+542.g31f379e-py3-none-any/polymerist/polymerist.py'> is not a package
True
polymerist.genutils
True
/home/timber/Documents/Python/polymerist_dev/polymerist/build/__editable__.polymerist-1.0.0+542.g31f379e-py3-none-any/polymerist/genutils
/home/timber/Documents/Python/polymerist_dev/polymerist/build/__editable__.polymerist-1.0.0+542.g31f379e-py3-none-any/polymerist/genutils/typetools
True
polymerist.duration
False
/home/timber/Documents/Python/polyme

# Development of miscellaneous new features for polymerist

### Test for paths

In [ ]:
from polymerist.genutils.filters import ALWAYS_FALSE_FILTER, ALWAYS_TRUE_FILTER
from polymerist.genutils.treetools.treecopy import copy_tree
from polymerist.genutils.treetools.treeinter import AbstractNodeCorrespondence, compile_tree_factory

from typing import Iterable, Optional
from anytree import Node
from pathlib import Path
from polymerist.genutils.decorators.functional import allow_string_paths

class PathToNodeCorrespondence(AbstractNodeCorrespondence, FROMTYPE=Path):
    '''Concrete implementation of how to produce filetrees from pathlib Paths'''
    def name(self, path : Path) -> str:
        return path.name
    
    def has_children(self, path : Path) -> bool:
        return path.is_dir()
    
    def children(self, path) -> Iterable[Path]:
        return path.iterdir()
    
make_file_tree = allow_string_paths(compile_tree_factory(PathToNodeCorrespondence(), class_alias='directory', obj_attr_name='file'))
# make_file_tree = compile_tree_factory(PathToNodeCorrespondence(), treename='directory')


@allow_string_paths # an explicit implementation just to see what an AbstractNodeCorrespondence simplifies
def file_tree(path : Path, max_depth : Optional[int]=None, _curr_depth : int=0) -> Node:
    '''Compiled a directory tree from a toplevel path. 
    
    Any subdirectories encountered will be expanded into their own tree,
    up to the specified maximum depth or until exhaustion if max_depth=None'''
    path_node = Node(
        name=path.name,
        file_path=path, # NOTE: can't name this attribute "path", as "path" is already an attribute fo the base Node class
    )

    if path.is_dir() and ( # recursively add subnodes IFF
            (max_depth is None)             # 1) no depth limit is set, or
            or (_curr_depth < max_depth)    # 2) a limit IS set, but hasn't been reached yet
        ): 
        for subpath in path.iterdir():
            subpath_node = file_tree(subpath, max_depth=max_depth, _curr_depth=_curr_depth+1)
            subpath_node.parent = path_node

    return path_node

In [ ]:
import polymerist as ps
from anytree.render import RenderTree, ContRoundStyle

# main_path = Path('.')
main_path = Path(ps.__file__).parent
main_path = str(main_path)
ft = make_file_tree(main_path, max_depth=None, exclude=lambda path : path.is_file() or path.name == '__pycache__')
# ct = copy_tree(ft, stop=lambda node : node.file.is_file() or node.file.name == '__pycache__')

rt_file = RenderTree(ft, style=ContRoundStyle())
print(rt_file.by_attr('name'))

### Unit tests

In [ ]:
from polymerist.genutils.attrs import compile_argfree_getable_attrs

class Test():
    '''Dummy class for testing that dynamic attribute inspection works properly'''
    FOO = 'bar'

    def __init__(self, answer : int=42, spam : str='eggs') -> None:
        self.answer = answer
        self.spam = spam

    @classmethod
    def get_foo(cls) -> str:
        return cls.FOO
    
    def get_answer(self) -> int:
        return self.answer

    def _get_spam(self) -> int:
        return self.spam

def test_compile_argfree_getable_attrs():
    t = Test()

In [ ]:
t = Test(answer=56)

compile_argfree_getable_attrs(t, getter_str='get')

### Test for Python modules

In [ ]:
import polymerist as ps
from polymerist import genutils, polymerist, duration
from polymerist.genutils import bits, typetools, importutils
from polymerist.genutils.decorators.functional import allow_string_paths

import json, math
import pkgutil, importlib

print(importutils.module_hierarchy(json))

# from anytree import RenderTree, Node, AsciiStyle, ContStyle, ContRoundStyle, DoubleStyle

In [ ]:
from types import ModuleType
from typing import Iterable, Optional, TypeAlias, Union

import inspect
import pkgutil
import importlib, importlib.machinery
from importlib.machinery import SourceFileLoader, FileFinder

from pathlib import Path
from anytree import Node


# EXTRACTING INFO FROM A SINGLE IMPORTED MODULE
def flexible_module_pass(module : Union[str, Path, ModuleType]) -> ModuleType:
    '''Flexible interface for supplying a ModuleType object as an argument
    Allows for passing a name (either module name or string path), Path location, or a module proper'''
    if isinstance(module, ModuleType):
        return module
    elif isinstance(module, str):
        raise NotImplementedError
    elif isinstance(module, Path):
        raise NotImplementedError
    else:
        raise TypeError(f'Cannot interpret object of type "{type(module).__name__}" as a module')

def is_package(module : ModuleType) -> bool:
    '''Check whether a Python module is a package (i.e. contains other importable modules within itself)'''
    # module_spec = getattr(module, '__spec__', None) # this doesn't work when a string is passed in
    module_spec = importlib.util.find_spec(module.__name__)
    if module_spec is None:
        raise ValueError(f'No ModuleSpec found for {module}')

    module_loader = module_spec.loader
    # module_loader = pkgutil.get_loader(module) # NOTE: while more compact than above, this function is slated for deprecation
    if module_loader is None:
        raise ValueError(f'No SourcefileLoader found for {module}')

    return module_loader.is_package(module.__name__)

# TODO : find way to get depth of submodule in toplevel ("number of dots" before standalone name)
def relative_module_name(module : ModuleType, relative_to : Optional[ModuleType]=None, remove_leading_dot : bool=True) -> str:
    '''Gets the name of a module relative to another (presumably toplevel) module
    If the given module is not in the path of the toplevel module, will simply return as module.__name__'''
    rel_mod_name = module.__name__
    if relative_to is not None:
        toplevel_prefix = relative_to.__name__
        if remove_leading_dot:
            toplevel_prefix += '.' # append dot to prefix to remove it later
        rel_mod_name = rel_mod_name.removeprefix(toplevel_prefix)

    return rel_mod_name


# COMPILING MODULE TREES FOR FULL PACKAGES
def module_tree(module : ModuleType, blacklist : Optional[Iterable[str]]=None, relative_to : Optional[ModuleType]=None, max_depth : Optional[int]=None, _curr_depth : int=0) -> Optional[Node]:
    '''Create a tree for a module and all its submodules, to a set depth and with optional blacklisting by module name'''
    if blacklist is None:
        blacklist = []

    # TODO: figure out way to get loader (or FileFinder?) for toplevel
    module_is_pkg = is_package(module)
    module_name = relative_module_name(module, relative_to=relative_to, remove_leading_dot=True)

    module_node = Node(
        name=module_name,
        module=module,
    )
    if module_is_pkg and ( # recursively add subnodes IFF
            (max_depth is None)             # 1) no depth limit is set, or
            or (_curr_depth < max_depth)    # 2) a limit IS set, but hasn't been reached yet
        ):
        for (submodule_loader, submodule_name, sub_is_pkg) in pkgutil.iter_modules(module.__path__):
            if submodule_name not in blacklist: # TOSELF: also worth checking the full __name__? (requires importing a potentially blacklisted module which isn't great)
                submodule = importlib.import_module(f'.{submodule_name}', package=module.__package__)
                submodule_node = module_tree(submodule, blacklist=blacklist, relative_to=module, max_depth=max_depth, _curr_depth=_curr_depth+1)
                submodule_node.parent = module_node

    return module_node


In [ ]:
import json, math
mods = (
    ps,
    polymerist, 
    genutils,
    duration,
    bits,
    typetools,
    json,
    math,
)

for mod in mods:
    print(mod.__name__, is_package(mod))
    print('\t', getattr(mod, '__file__', None), getattr(mod, '__path__', None))
    mspec = mod.__spec__
    par_mod = importlib.find_loader(mspec.parent).load_module()
    print('\t', mspec.origin, mspec.parent, mspec.name, relative_module_name(mod, relative_to=par_mod))

In [ ]:
from anytree import RenderTree, Node, AsciiStyle, ContStyle, ContRoundStyle, DoubleStyle


mt = module_tree(ps, blacklist=('decorators',), max_depth=None, relative_to=ps)
cmt = copy_tree(mt, stop=lambda node : not is_package(node.module))
rt_mod = RenderTree(cmt)
rt_mod_str = rt_mod.by_attr('name')

with Path('packages.txt').open('w') as file:
    file.write(rt_mod_str)
print(rt_mod_str)

In [ ]:
copm = copy_tree(mt, stop=lambda node : not is_package(node.module), attr_filter=lambda attr : attr != 'module')
print(RenderTree(copm).by_attr('name'))

In [ ]:
copf = copy_tree(ft, attr_filter=lambda attr : attr != 'file')
print(RenderTree(copf).by_attr('name'))

In [ ]:
from anytree.exporter import DictExporter, DotExporter, JsonExporter

dexp = DictExporter()
jexp =JsonExporter(indent=4, sort_keys=True)
mod_dict = dexp.export(copm)
fil_dict = dexp.export(copf)

In [ ]:
linem = set(l for l in DotExporter(copm))
linef = set(l for l in DotExporter(copf))

In [ ]:
rt_file_str = rt_file.by_attr('name')
with Path('directories.txt').open('w') as file:
    file.write(rt_file_str)
print(rt_file_str)

In [ ]:
from polymerist import tests

In [ ]:
import importlib.resources


p = importlib.resources.files(tests)
p.exists()

In [ ]:
from polymerist import data

In [ ]:
[f.name for f in p.iterdir()]

In [ ]:
from anytree.iterators import LevelOrderIter, LevelOrderGroupIter

cft = copy_tree(ft, stop=lambda node : node.file.is_file())
cft

root_path = Path(cft.file) # need to make a clopy of this, as the root node's path will be mutated by the subsequent resolution
for node in LevelOrderIter(cft):
    node.file = node.file.relative_to(root_path)
    
    name = node.file.name
    if node.parent is not None:
        node.file = node.parent.file / f'{name}'
    
    asb_path = Path('polymerist/polymerist/tests') / node.file
    asb_path.mkdir(exist_ok=True)

    if node.is_root:
        continue
    init_path = asb_path / '__init__.py'
    with init_path.open('w') as file:
        file.write(f"'''Unit tests for `{name}` package'''")

In [ ]:
from polymerist.genutils.fileutils import pathutils

In [ ]:
root_path.with_name('foo')

In [ ]:
print(RenderTree(cft))

In [ ]:
main_path = Path(ps.__file__).parent
filetree = file_tree_from_path(main_path, max_depth=None)

rt = RenderTree(filetree, style=ContRoundStyle())
print(rt.by_attr('name'))

## Testing polymerist importability within environment

In [ ]:
import numpy as np
from openff.toolkit import Molecule, Topology, ForceField

In [ ]:
import polymerist as ps
from polymerist.genutils import pyimports, importutils

import pandas as pd
print(importutils.module_hierarchy(ps))

In [ ]:
import nglview

print(nglview.__version__)
nglview.demo()

In [ ]:
from polymerist.polymers.monomers import specification
from rdkit import Chem

smi = 'CCO-c1ccccc1-N=C=C'
mol1 = Chem.MolFromSmiles(smi)
display(mol1)

sma = specification.expanded_SMILES(smi, assign_map_nums=True)
exp_sma = specification.compliant_mol_SMARTS(sma)
mol2 = Chem.MolFromSmarts(sma)
display(mol2)


In [ ]:
from openff.toolkit import Molecule

offmol = Molecule.from_smiles(smi)
offmol.generate_conformers(n_conformers=1)
offmol.visualize(backend='nglview')

## Dynamically reading all import statements in codebase

In [ ]:
import polymerist as ps
from polymerist.genutils import pyimports, importutils

print(importutils.module_hierarchy(ps))

In [ ]:
infos = pyimports.extract_imports_from_module(ps)

df = pd.DataFrame.from_records([info.__dict__ for info in infos])
df.to_csv('test.csv')

In [ ]:
nonrel = [info for info in infos if not info.is_relative and info.parent_module is None]
len(nonrel)

In [ ]:
import sys

imported_names = set(info.object_name for info in nonrel)
imported_names

registered_builtins = set(sys.builtin_module_names)
registered_stdlibs = set(sys.stdlib_module_names)

nb_imports = imported_names - registered_builtins - registered_stdlibs
nb_imports

# Another thing

In [ ]:
import nglview

nglview.demo()